In [65]:
import requests
import numpy as np
import pandas as pd
import os

## In this we will be:
- Requesting data from census.gov
- Cleaning, combining and reorganizing the data
- Inserting some calculations
- Exporting to Excel

## Requesting data

In [66]:
# This will be public so census key is hidden
cwd = os.getcwd()
key_path = cwd + "\Census Key.txt"
with open(key_path, 'r') as file:
    key = file.read()

In [67]:
host = "https://api.census.gov/data"
dataset = "acs/acs1/profile"
get = "?get="
variables = "NAME,DP05_0001E,DP03_0026E,DP03_0024E"
location = "&for=state:*"


In [68]:
r = requests.get(f"{host}/2019/{dataset}{get}{variables}{location}{key}")
census_db_2019 = r.json()

r = requests.get(f"{host}/2021/{dataset}{get}{variables}{location}{key}")
census_db_2021 = r.json()


## Cleaning, combining and reorganizing

In [69]:
# converting raw data to dataframe. 
# Renaming the columns, sorting the data and dropping row 0 as row 0 was the column headers when pulled from API
census_db_2019 = pd.DataFrame(census_db_2019)
census_db_2019 = census_db_2019.rename(columns = {0: "state", 1: "total_pop", 2: "employed_pop", 3: "wfh_pop", 4: "state_code"})
census_db_2019 = census_db_2019.drop(0)
census_db_2019.reset_index(inplace=True, drop=True)
census_db_2019.sort_values("state_code", inplace=True)
census_db_2019.head(5)

,state,total_pop,employed_pop,wfh_pop,state_code
44,Alabama,4903185,2153467,72440,01
45,Alaska,731545,338011,15141,02
46,Arizona,7278717,3305302,248326,04
47,Arkansas,3017804,1325091,46196,05
48,California,39512223,19078101,1188387,06


In [70]:
# converting raw data to dataframe. 
# Renaming the columns, sorting the data and dropping row 0 as row 0 was the column headers when pulled from API
census_db_2021 = pd.DataFrame(census_db_2021)
census_db_2021 = census_db_2021.rename(columns = {0: "state", 1: "total_pop", 2: "employed_pop", 3: "wfh_pop", 4: "state_code"})
census_db_2021 = census_db_2021.drop(0)
census_db_2021.reset_index(inplace=True, drop=True)
census_db_2021.sort_values("state_code", inplace=True)
census_db_2021.head(5)

,state,total_pop,employed_pop,wfh_pop,state_code
0,Alabama,5039877,2190915,208767,01
51,Alaska,732673,327953,35462,02
2,Arizona,7276316,3314799,680459,04
3,Arkansas,3025891,1323511,127385,05
4,California,39237836,18156051,3812070,06


In [71]:
# combining the dataframes
census_db = pd.concat([census_db_2019,census_db_2021], axis= 0, keys=[2019, 2021])
census_db["year"] = census_db.index.get_level_values(0)

census_db

state total_pop employed_pop  wfh_pop state_code  year
2019 44        Alabama   4903185      2153467    72440         01  2019
     45         Alaska    731545       338011    15141         02  2019
     46        Arizona   7278717      3305302   248326         04  2019
     47       Arkansas   3017804      1325091    46196         05  2019
     48     California  39512223     19078101  1188387         06  2019
...                ...       ...          ...      ...        ...   ...
2021 47     Washington   7738692      3696564   887544         53  2021
     48  West Virginia   1782959       721479    72269         54  2021
     49      Wisconsin   5895908      2991136   437295         55  2021
     50        Wyoming    578803       287432    25382         56  2021
     1     Puerto Rico   3263584      1085614    78333         72  2021

[104 rows x 6 columns]

In [72]:
# changing datatypes of everything but "state" to int
for column in (census_db.columns.to_list())[1:]:
    census_db[column] = pd.to_numeric(census_db[column])

census_db.dtypes


state           object
total_pop        int64
employed_pop     int64
wfh_pop          int64
state_code       int64
year             int64
dtype: object

In [73]:
# The census represents missing data with a negative number such as -9999999. We are replacing data that may be missing here.
for column in (census_db.columns.to_list())[1:]:
    census_db[column] = census_db[column].apply(lambda x: x if x >= 0 else np.nan)


In [74]:
# chaning place of columns
census_db = census_db[["state"] + ["state_code"] + ["year"] + ["total_pop"] + ["employed_pop"] + ["wfh_pop"]]

census_db

state  state_code  year  total_pop  employed_pop  wfh_pop
2019 44        Alabama           1  2019    4903185       2153467    72440
     45         Alaska           2  2019     731545        338011    15141
     46        Arizona           4  2019    7278717       3305302   248326
     47       Arkansas           5  2019    3017804       1325091    46196
     48     California           6  2019   39512223      19078101  1188387
...                ...         ...   ...        ...           ...      ...
2021 47     Washington          53  2021    7738692       3696564   887544
     48  West Virginia          54  2021    1782959        721479    72269
     49      Wisconsin          55  2021    5895908       2991136   437295
     50        Wyoming          56  2021     578803        287432    25382
     1     Puerto Rico          72  2021    3263584       1085614    78333

[104 rows x 6 columns]

## Calculations

In [75]:
# calculating the work from home %
census_db["wfh_percent"] = round(census_db["wfh_pop"] / census_db["employed_pop"] *100, 2)
census_db

C:\Users\DannyP\AppData\Local\Temp\ipykernel_24348\3810875355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_db["wfh_percent"] = round(census_db["wfh_pop"] / census_db["employed_pop"] *100, 2)


state  state_code  year  total_pop  employed_pop  wfh_pop  \
2019 44        Alabama           1  2019    4903185       2153467    72440   
     45         Alaska           2  2019     731545        338011    15141   
     46        Arizona           4  2019    7278717       3305302   248326   
     47       Arkansas           5  2019    3017804       1325091    46196   
     48     California           6  2019   39512223      19078101  1188387   
...                ...         ...   ...        ...           ...      ...   
2021 47     Washington          53  2021    7738692       3696564   887544   
     48  West Virginia          54  2021    1782959        721479    72269   
     49      Wisconsin          55  2021    5895908       2991136   437295   
     50        Wyoming          56  2021     578803        287432    25382   
     1     Puerto Rico          72  2021    3263584       1085614    78333   

         wfh_percent  
2019 44         3.36  
     45         4.48  
     46         7.51  
     47         3.49  
     48         6.23  
...              ...  
2021 47        24.01  
     48        10.02  
     49        14.62  
     50         8.83  
     1          7.22  

[104 rows x 7 columns]

## To Excel

In [76]:
census_db.to_excel("Census_WFH.xlsx", index=False)